# Data Loading

## Objectives of Notebook

This notebook aims to:

* Create dimension & fact tables.
* Load data to data warehouse.

## Import Libraries

In [1]:
%autosave 60
%load_ext autoreload
%autoreload 2

Autosaving every 60 seconds


In [2]:
import pandas as pd

## Import Customised Functions from Source Code

In [3]:
# Import Functions for Utilities
from src.utils import  load_dw

# Import Functions for train & predict
from src.train import out_dim_date, out_dim_model, out_dim_type, out_dim_breakdown, out_dim_cov_breakdown, out_fct_forecast, out_fct_actual, out_fct_breakdown, out_fct_cov_breakdown

## Import Preprocessed Data

* Data that have been preprocessed on previous notebook.

In [4]:
# Load previous DataFrame from previous notebook
%store -r forecast_merge
%store -r nowcast_merge

%store -r debit_breakdown
%store -r credit_breakdown
%store -r ue_bank_breakdown
%store -r ue_nonbank_breakdown
%store -r digital_banking_breakdown
%store -r nowcast_breakdown_merge

In [5]:
# Create a list for all electronic transaction breakdown DataFrames
transaction_breakdown = [debit_breakdown, credit_breakdown, ue_bank_breakdown, ue_nonbank_breakdown, digital_banking_breakdown]

# Merge all electronic transaction breakdown DataFrames into one DataFrame
transaction_breakdown_merge = pd.concat(transaction_breakdown)

# Reset DateTime index for dimension table creation
transaction_breakdown_merge.reset_index(inplace=True)

# Show transaction_breakdown_merge DataFrame
transaction_breakdown_merge

,date,type,breakdown,transaction_volume,transaction_nominal
0,2019-01-01,Debit,Tunai,3.440193e+08,2.570000e+14
1,2019-01-02,Debit,Tunai,3.381152e+09,2.530000e+14
2,2019-01-03,Debit,Tunai,3.639403e+09,2.570000e+14
3,2019-01-04,Debit,Tunai,3.522287e+09,2.520000e+14
4,2019-01-05,Debit,Tunai,3.861731e+09,3.110000e+14
...,...,...,...,...,...
1807,2023-01-05,Digital Banking,Internet Banking,4.479419e+08,3.990000e+15
1808,2023-01-06,Digital Banking,Internet Banking,4.719213e+08,3.580000e+15
1809,2023-01-07,Digital Banking,Internet Banking,4.833853e+08,3.960000e+15
1810,2023-01-08,Digital Banking,Internet Banking,2.090318e+08,3.650000e+15


## Dimension Tables Creation

#### dim_date

In [6]:
# Combine 'date' column from forecast_merge and nowcast_merge into one dataframe
date_com = pd.concat([forecast_merge['date'], nowcast_merge['date']]).drop_duplicates().reset_index(drop=True)

# Call out_dim_date function to create 'dim_date' dataframe
dim_date = out_dim_date(df=date_com)

# Show 'dim_date' dataframe
dim_date

,date,year,quarter,month,month_name,day,day_name
0,2019-01-01,2019,1,1,January,1,Tuesday
1,2019-02-01,2019,1,2,February,4,Friday
2,2019-03-01,2019,1,3,March,4,Friday
3,2019-04-01,2019,2,4,April,0,Monday
4,2019-05-01,2019,2,5,May,2,Wednesday
...,...,...,...,...,...,...,...
64,2024-05-01,2024,2,5,May,2,Wednesday
65,2024-06-01,2024,2,6,June,5,Saturday
66,2024-07-01,2024,3,7,July,0,Monday
67,2024-08-01,2024,3,8,August,3,Thursday


#### dim_model

In [7]:
# Combine 'model' column from forecast_merge and nowcast_merge into one dataframe
model_com = pd.concat([forecast_merge['model'], nowcast_merge['model']]).drop_duplicates().reset_index(drop=True)

# Call out_dim_model function to create 'dim_model' dataframe
dim_model = out_dim_model(df=model_com, model_col='model')

# Show 'dim_model' dataframe
dim_model

,model,model_category
model_id,,
0,Actual,NA
2,Prophet,Statistical
1,Auto-ARIMA,Statistical
3,Exponential Smoothing,Statistical
4,Bayesian Ridge,ML
5,LSTM,ML


#### dim_type

In [8]:
# Combine 'type' column from forecast_merge and nowcast_merge into one dataframe
type_com = pd.concat([forecast_merge['type'], nowcast_merge['type']]).drop_duplicates().reset_index(drop=True)

# Call out_dim_type function to create 'dim_type' dataframe
dim_type = out_dim_type(df=type_com, type_col='type')

# Show 'dim_type' dataframe
dim_type

,type,type_category
type_id,,
11,Debit,Card
12,Credit,Card
13,UE Bank,Card
23,UE Non-Bank,Non-card
24,QRIS,Non-card
21,Digital Banking,Non-card
1,Google Trends,External
2,Electronic Transaction Nom,Internal
3,Combined,Combination


#### | dim_breakdown

In [9]:
# Call out_dim_breakdown function to create a dim_breakdown dimesnion table from the transaction_breakdown_merge DataFrame
dim_breakdown = out_dim_breakdown(df=transaction_breakdown_merge, breakdown_col='breakdown')

# Show dim_breakdown DataFrame
dim_breakdown

,breakdown
breakdown_id,
1,Tunai
2,Non Tunai
3,Belanja
4,Setor Tunai
5,Pembayaran
6,Transfer Intra
7,Transfer Inter
8,Transaksi Online
9,Transfer


In [10]:
# Call out_dim_cov_breakdown function to create a dim_cov_breakdown dimesnion table from the nowcast_breakdown_merge DataFrame
dim_cov_breakdown = out_dim_cov_breakdown(df=nowcast_breakdown_merge, cov_breakdown_col='covariates')

# Show dim_cov_breakdown DataFrame
dim_cov_breakdown

,covariates
cov_breakdown_id,
5,microcars & city cars
39,electricity
6,motor vehicle & parts
31,food and beverages
28,cooking & recipe
48,cleaning agents
1,motorcycle
26,non alcoholic beverages
30,restaurant


### Fact tables creation

#### fct_forecast

In [11]:
# Create a fact table from the the forecasted values in the forecast_merge DataFrame
fct_forecast = out_fct_forecast(df=forecast_merge,
                                    fct_col_id='forecast_id',
                                    model_col='model',
                                    type_col='type'
)

# Show fct_forecast DataFrame
fct_forecast

component,forecast_id,date,transaction_volume,mape_vol,rmse_vol,mae_vol,model_score_vol,yoy_growth_vol,mom_growth_vol,transaction_nominal,mape_nom,rmse_nom,mae_nom,model_score_nom,yoy_growth_nom,mom_growth_nom,type_id,model_id
57,2022-09-01_11,2022-09-01,5.876562e+08,0.03,25470327.0,21141962.0,14.0,NaN,NaN,6.021450e+14,0.04,3.405061e+13,2.769318e+13,9.3,NaN,NaN,11,2
58,2022-10-01_11,2022-10-01,6.842797e+08,0.03,25470327.0,21141962.0,14.0,NaN,0.1644,6.714065e+14,0.04,3.405061e+13,2.769318e+13,9.3,NaN,0.1150,11,2
59,2022-11-01_11,2022-11-01,6.312889e+08,0.03,25470327.0,21141962.0,14.0,NaN,-0.0774,6.587047e+14,0.04,3.405061e+13,2.769318e+13,9.3,NaN,-0.0189,11,2
60,2022-12-01_11,2022-12-01,6.618328e+08,0.03,25470327.0,21141962.0,14.0,NaN,0.0484,6.925998e+14,0.04,3.405061e+13,2.769318e+13,9.3,NaN,0.0515,11,2
61,2023-01-01_11,2023-01-01,6.472535e+08,0.03,25470327.0,21141962.0,14.0,NaN,-0.0220,6.770033e+14,0.04,3.405061e+13,2.769318e+13,9.3,NaN,-0.0225,11,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2024-05-01_21,2024-05-01,1.627957e+09,0.03,48505583.0,38610993.0,2.6,0.2716,0.0203,4.918310e+15,0.04,2.580744e+14,1.912362e+14,13.5,0.1330,-0.0570,21,3
78,2024-06-01_21,2024-06-01,1.661299e+09,0.03,48505583.0,38610993.0,2.6,0.2388,0.0205,5.257487e+15,0.04,2.580744e+14,1.912362e+14,13.5,0.0650,0.0690,21,3
79,2024-07-01_21,2024-07-01,1.692410e+09,0.03,48505583.0,38610993.0,2.6,0.2164,0.0187,5.274736e+15,0.04,2.580744e+14,1.912362e+14,13.5,0.0876,0.0033,21,3
80,2024-08-01_21,2024-08-01,1.720598e+09,0.03,48505583.0,38610993.0,2.6,0.1923,0.0167,5.452359e+15,0.04,2.580744e+14,1.912362e+14,13.5,0.0684,0.0337,21,3


#### fct_actual_forecast

In [12]:
# Create a fact table from all the actual values in the forecast_merge DataFrame
fct_actual_forecast = out_fct_actual(df=forecast_merge,
                                    fct_col_id='forecast_id',
                                    model_col='model',
                                    type_col='type'
)

# Show fct_actual DataFrame
fct_actual_forecast

component,forecast_id,date,transaction_volume,mape_vol,rmse_vol,mae_vol,model_score_vol,yoy_growth_vol,mom_growth_vol,transaction_nominal,mape_nom,rmse_nom,mae_nom,model_score_nom,yoy_growth_nom,mom_growth_nom,type_id,model_id
0,2019-01-01_11,2019-01-01,5.521845e+08,NaN,NaN,NaN,NaN,NaN,NaN,5.480000e+14,NaN,NaN,NaN,NaN,NaN,NaN,11,0
1,2019-02-01_11,2019-02-01,5.439533e+08,NaN,NaN,NaN,NaN,NaN,-0.0149,5.310000e+14,NaN,NaN,NaN,NaN,NaN,-0.0310,11,0
2,2019-03-01_11,2019-03-01,5.858424e+08,NaN,NaN,NaN,NaN,NaN,0.0770,5.590000e+14,NaN,NaN,NaN,NaN,NaN,0.0527,11,0
3,2019-04-01_11,2019-04-01,5.638822e+08,NaN,NaN,NaN,NaN,NaN,-0.0375,5.420000e+14,NaN,NaN,NaN,NaN,NaN,-0.0304,11,0
4,2019-05-01_11,2019-05-01,6.068453e+08,NaN,NaN,NaN,NaN,NaN,0.0762,6.210000e+14,NaN,NaN,NaN,NaN,NaN,0.1458,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,2023-05-01_21,2023-05-01,1.325064e+09,NaN,NaN,NaN,NaN,0.4692,0.0239,5.010000e+15,NaN,NaN,NaN,NaN,0.3289,0.1208,21,0
53,2023-06-01_21,2023-06-01,1.361199e+09,NaN,NaN,NaN,NaN,0.4288,0.0273,4.600000e+15,NaN,NaN,NaN,NaN,0.0698,-0.0818,21,0
54,2023-07-01_21,2023-07-01,1.413664e+09,NaN,NaN,NaN,NaN,0.3523,0.0385,5.040000e+15,NaN,NaN,NaN,NaN,0.1560,0.0957,21,0
55,2023-08-01_21,2023-08-01,1.428076e+09,NaN,NaN,NaN,NaN,0.3718,0.0102,5.100000e+15,NaN,NaN,NaN,NaN,0.1184,0.0119,21,0


#### fct_brekdown

In [13]:
# Create a fact table from the value in transaction_breakdown_merge DataFrame
fct_breakdown = out_fct_breakdown(df=transaction_breakdown_merge,
                                  fct_col_id='transaction_breakdown_id',
                                  breakdown_col='breakdown',
                                  type_col='type'
)

# Show fct_breakdown DataFrame
fct_breakdown

,transaction_breakdown_id,date,transaction_volume,transaction_nominal,breakdown_id,type_id
0,2019-01-01_11,2019-01-01,3.440193e+08,2.570000e+14,1,11
1,2019-01-02_11,2019-01-02,3.381152e+09,2.530000e+14,1,11
2,2019-01-03_11,2019-01-03,3.639403e+09,2.570000e+14,1,11
3,2019-01-04_11,2019-01-04,3.522287e+09,2.520000e+14,1,11
4,2019-01-05_11,2019-01-05,3.861731e+09,3.110000e+14,1,11
...,...,...,...,...,...,...
1807,2023-01-05_21,2023-01-05,4.479419e+08,3.990000e+15,18,21
1808,2023-01-06_21,2023-01-06,4.719213e+08,3.580000e+15,18,21
1809,2023-01-07_21,2023-01-07,4.833853e+08,3.960000e+15,18,21
1810,2023-01-08_21,2023-01-08,2.090318e+08,3.650000e+15,18,21


#### fct_nowcast

In [14]:
# Create a fact table fct_nowcast for all the nowcasted values in the nowcast_merge DataFrame
fct_nowcast = out_fct_forecast(df=nowcast_merge, 
                                fct_col_id = 'nowcast_id', 
                                model_col = 'model',
                                type_col = 'type')

# Show fct_forecast DataFrame
fct_nowcast

component,nowcast_id,date,household_consumption,mape,rmse,mae,model_score,yoy_growth,mom_growth,type_id,model_id
40,2021-12-01_1,2021-12-01,2.396035e+15,0.02,7.795844e+13,6.204314e+13,4.2,NaN,NaN,1,2
41,2022-01-01_1,2022-01-01,2.437056e+15,0.02,7.795844e+13,6.204314e+13,4.2,NaN,0.0171,1,2
42,2022-02-01_1,2022-02-01,2.494100e+15,0.02,7.795844e+13,6.204314e+13,4.2,NaN,0.0234,1,2
43,2022-03-01_1,2022-03-01,2.411883e+15,0.02,7.795844e+13,6.204314e+13,4.2,NaN,-0.0330,1,2
44,2022-04-01_1,2022-04-01,2.563282e+15,0.02,7.795844e+13,6.204314e+13,4.2,NaN,0.0628,1,2
...,...,...,...,...,...,...,...,...,...,...,...
57,2023-05-01_3,2023-05-01,2.793461e+15,0.12,2.808986e+14,2.721295e+14,16.6,0.2167,0.1717,3,5
58,2023-06-01_3,2023-06-01,2.794524e+15,0.12,2.808986e+14,2.721295e+14,16.6,0.2228,0.0004,3,5
59,2023-07-01_3,2023-07-01,2.788723e+15,0.12,2.808986e+14,2.721295e+14,16.6,0.2120,-0.0021,3,5
60,2023-08-01_3,2023-08-01,2.798550e+15,0.12,2.808986e+14,2.721295e+14,16.6,0.2114,0.0035,3,5


#### fct_actual_forecast

In [15]:
# Create fact tables fct_actual_nowcast from all the actual values in the nowcast_merge DataFrame
fct_actual_nowcast = out_fct_actual(df=nowcast_merge, 
                                        fct_col_id = 'nowcast_id', 
                                        model_col = 'model',
                                        type_col = 'type')

# Show fct_actual DataFrame
fct_actual_nowcast

component,nowcast_id,date,household_consumption,mape,rmse,mae,model_score,yoy_growth,mom_growth,type_id,model_id
0,2020-01-01_1,2020-01-01,2.280000e+15,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,2020-02-01_1,2020-02-01,2.224206e+15,NaN,NaN,NaN,NaN,NaN,-0.0245,1,0
2,2020-03-01_1,2020-03-01,2.165930e+15,NaN,NaN,NaN,NaN,NaN,-0.0262,1,0
3,2020-04-01_1,2020-04-01,2.130000e+15,NaN,NaN,NaN,NaN,NaN,-0.0166,1,0
4,2020-05-01_1,2020-05-01,2.143209e+15,NaN,NaN,NaN,NaN,NaN,0.0062,1,0
...,...,...,...,...,...,...,...,...,...,...,...
35,2022-12-01_3,2022-12-01,2.667785e+15,NaN,NaN,NaN,NaN,0.1107,0.0040,3,0
36,2023-01-01_3,2023-01-01,2.680000e+15,NaN,NaN,NaN,NaN,0.1074,0.0046,3,0
37,2023-02-01_3,2023-02-01,2.700690e+15,NaN,NaN,NaN,NaN,0.0995,0.0077,3,0
38,2023-03-01_3,2023-03-01,2.732798e+15,NaN,NaN,NaN,NaN,0.0953,0.0119,3,0


#### fct_cov_breakdown

In [16]:
# Create a fact table from the value in nowcast_breakdown_merge DataFrame
fct_cov_breakdown = out_fct_cov_breakdown(df=nowcast_breakdown_merge,
                                          fct_col_id='nowcast_breakdown_id',
                                          type_col='type',
                                          model_col='model',
                                          cov_breakdown_col='covariates',
)
# Show fct_cov_breakdown DataFrame
fct_cov_breakdown

,nowcast_breakdown_id,date,correlation_coefficient,type_id,model_id,cov_breakdown_id
0,2020-01-01_5,2020-01-01,0.77,1,0,5
1,2020-02-01_5,2020-02-01,0.77,1,0,5
2,2020-03-01_5,2020-03-01,0.77,1,0,5
3,2020-04-01_5,2020-04-01,0.77,1,0,5
4,2020-05-01_5,2020-05-01,0.77,1,0,5
...,...,...,...,...,...,...
4273,2023-05-01_54,2023-05-01,0.72,3,5,54
4274,2023-06-01_54,2023-06-01,0.72,3,5,54
4275,2023-07-01_54,2023-07-01,0.72,3,5,54
4276,2023-08-01_54,2023-08-01,0.72,3,5,54


### Data Warehouse Loading

In [17]:
# Call load_dw function to establish connection to the data warehouse
load_conn = load_dw(server=r'LAPTOP-14FUNTO5\NEWSERVER;', dw_name='dksp_dashboard')

Successfully connected to the data warehouse.


In [18]:
# Load dim_date DataFrame into the data warehouse
dim_date.to_sql('dim_date', schema='dbo', con=load_conn, if_exists='replace')

# Load dim_model DataFrame into the data warehouse
dim_model.to_sql('dim_model', schema='dbo', con=load_conn, if_exists='replace')

# Load dim_type DataFrame into the data warehouse
dim_type.to_sql('dim_type', schema='dbo', con=load_conn, if_exists='replace')

# Load dim_breakdown DataFrame to the data warehouse
dim_breakdown.to_sql('dim_breakdown', schema='dbo', con=load_conn, if_exists='replace')

# Load dim_cov_breakdown DataFrame to the data warehouse
dim_cov_breakdown.to_sql('dim_cov_breakdown', schema='dbo', con=load_conn, if_exists='replace')

# Load fct_forecast DataFrame to the data warehouse
fct_forecast.to_sql('fct_forecast', schema='dbo', con=load_conn, if_exists='replace')

# Load fct_actual DataFrame to the data warehouse
fct_actual_forecast.to_sql('fct_actual_forecast', schema='dbo', con=load_conn, if_exists='replace')

# Load fct_breakdown DataFrame to the data warehouse
fct_breakdown.to_sql('fct_breakdown', schema='dbo', con=load_conn, if_exists='replace')

# Load fct_forecast DataFrame into the data warehouse
fct_nowcast.to_sql('fct_nowcast', schema='dbo', con=load_conn, if_exists='replace')

# Load fct_actual DataFrame into the data warehouse
fct_actual_nowcast.to_sql('fct_actual_nowcast', schema='dbo', con=load_conn, if_exists='replace')

# Load fct_cov_breakdown DataFrame into the data warehouse
fct_cov_breakdown.to_sql('fct_cov_breakdown', schema='dbo', con=load_conn, if_exists='replace')

156